# Consistent Bayes: Canonical Epidemiological Model (Parallelized)
---

Copyright 2018 Michael Pilosov


### Import Libraries


In [ ]:
# Mathematics and Plotting
from HelperFuns import * # pyplot wrapper functions useful for visualizations, numpy, scipy, etc.
# %matplotlib inline
# %matplotlib notebook
from cbayes import sample, solve, distributions
# Interactivity
from ipywidgets import *

import numpy as np
from scipy.integrate import odeint
# import matplotlib.pyplot as plt
import progressbar

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = 20, 10

In [ ]:
import ipyparallel as ipp # import the client
rc = ipp.Client()
dv = rc[:]
rc.ids # list the ids of the engine the client can communicate with

# Example Summary

---



---
## Define Functions

In [ ]:
##### FIXED PARAMETERS #####
num_days = 365
num_observations = 365 # how many observations are you able to make over the year?

assumed_population_size = 5E4 # your assumed population size (truth for the model)
infection_rate = 0.2 # (ppl/day) assumed infection rate
recovery_time = 10 # (1/days) assumed recovery rate. both of these shouldn't matter if we do a good job ID'ing them.
####

def SSE_generator(model, obs_data, sigma=1):   # this generates a sum of squared residuals.
    def QoI_fun(inputs): # that conforms to our desired model input
        M = len(obs_data)
        predictions = model(inputs)
        assert predictions.shape[1] == len(obs_data)
        residuals = predictions - obs_data
        QoI = (1./M)*np.sum( (residuals/sigma)**2, axis=1 )
        return QoI
    return QoI_fun

# The SIR model differential equations.
def deriv(y, t, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I 
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

def SIR(beta=infection_rate, gamma = 1./recovery_time, I0=1./assumed_population_size, t=np.linspace(0,num_days,num_observations),verbose=False):
    # I0 is number of initial infected individuals
    # R0 is number of initial  recovered/immune individuals
    # S0 is everyone else, the susceptible 
    # beta is contact rate (socialness of society) (ppl/day)
    # gamma is recovery rate (days^-1)
    R0 = 0 # WE ALSO ASSUME ZERO NATURAL IMMUNITY OR VACCINE
    S0 = 1. - R0 - I0
    try:
        assert(S0+I0+R0 == 1)
    except AssertionError:
        print("R0 must be nonnegative. you passed infeasible parameters")
        pass
    y0 = np.array([S0, I0, R0])     # Initial conditions vector
    if verbose:
        print("[b, g] = [%.2f, %.2f], [S0 I0 R0] = [%.2f, %1.2e, %1.2e]"%(beta, gamma, y0[0], y0[1], y0[2]), "| max time =", t[-1], "| h =", len(t))
    ret = odeint(deriv, y0, t, args=(beta, gamma))
    S, I, R = ret.T # Integrate the SIR equations over the time grid, t.
    return S, I, R


# This is for when we collect 2-D data
def epidemic_IR(lam, verbose=False):
    num_samples = lam.shape[0]
    days = num_days
    h = num_observations
    t = np.linspace(0, days, h)
    I_out = np.empty(shape=(num_samples, h))
    R_out = np.empty(shape=(num_samples, h))
    bar = progressbar.ProgressBar()
    for n in bar(range(num_samples)):
        beta, gamma = lam[n,:]
        I0 = 1./assumed_population_size
        S, I, R = SIR(beta, gamma, I0, t, verbose=verbose)
        I_out[n,:] = I
        R_out[n,:] = R
    return np.array([I_out, R_out])


def epidemic_I(lam, verbose=False): # necessary wrapper functions for SSE_generator
    I_out, R_out = epidemic_IR(lam,verbose)
    return I_out

def epidemic_R(lam, verbose=False):
    I_out, R_out = epidemic_IR(lam,verbose)
    return R_out

t = np.linspace(0, num_days, num_observations)
S,I,R = SIR(t=t,verbose=True)

## Time-Series Plot with Background Grid

In [ ]:
# # Plot the data on three separate curves for S(t), I(t) and R(t)
# fig = plt.figure(facecolor='w')
# ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
# ax.plot(t, S, 'b', alpha=0.5, lw=2, label='Susceptible')
# ax.plot(t, I, 'r', alpha=0.5, lw=2, label='Infected')
# ax.plot(t, R, 'g', alpha=0.5, lw=2, label='Recovered with immunity')

# ax.set_xlabel('Time (days)')
# ax.set_ylabel('Proportion of Population')

# ax.set_ylim(0,1.2)
# ax.yaxis.set_tick_params(length=0)
# ax.xaxis.set_tick_params(length=0)

# ax.grid(b=True, which='major', c='w', lw=2, ls='-')
# legend = ax.legend()
# legend.get_frame().set_alpha(0.5)
# for spine in ('top', 'right', 'bottom', 'left'):
#     ax.spines[spine].set_visible(False)
# plt.show()

## Get a sense of the variation 

A common parameter choice is $\beta$ = 0.2 and $\gamma$ = 0.1, let us see how much varation this leads to even if we fix these.

In [ ]:
n = 50
lam = np.random.rand(n,2)
lam[:,0] = 0.1+0.2*lam[:,0]# beta
lam[:,1] = 0.2*lam[:,1] # gamma

# lam[:,2] = 10**(-np.linspace(3, 5, n) ) # I0
# lam[:,2] = np.linspace(1./1E5, 1./1E4, n) # I0
data = epidemic_IR(lam,verbose=False)

In [ ]:
for i in range(n):
    plt.plot(t, data[0, i, :], alpha=5.0/n, c='r') # I
    plt.plot(t, data[1, i, :], alpha=5.0/n, c='k') # R
#     plt.plot(t, 1 - data[0, i, :] - data[1, i, :], c='b', alpha=2/n) # S
# plt.plot(t, 1-I-R, c='b', lw=3, ls=':')
plt.plot(t, I, c='r', lw=3, ls='-', label="Infected")
plt.plot(t, R, c='k', lw=3, ls='-', label="Recovered")
plt.xlabel('Time (days)', fontsize=28)
plt.ylabel('Proportion of Population', fontsize=28)
plt.xlim([0,365])
plt.ylim([0,1])
plt.legend(fontsize=18)
plt.savefig('saveimgs/SIR/SIR_signal.png')
plt.show()

--- 

# Define Your Parameter-to-Observables Map


In [ ]:
observables = 2 # are you observing I (num_obs = 1) or both I and R (num_obs = 2)
output_dim = 1 # are you assimilating the data into a 1D or 2D vector?
sd_R = .05
sd_I = .05

--- 

# Define Prior Measure $P_\Lambda$


In [ ]:
input_dim = 2 # Specify input space dimension (n) -- this is a handle that the widgets depend on.
num_samples = int(1E4) # number of input samples (N)
s_set = sample.sample_set(size=(num_samples, input_dim))
min_population_size = 1E4 # that you are willing to consider.
max_population_size = 1E5 # that you are willing to consider.

s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': 0.1, 'scale': 0.3}, dim=0) # beta - rate of (ppl/day)
s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': 0, 'scale': 0.2}, dim=1) # gamma - recovery rate (1/days)
# s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': 1./max_population_size, 'scale': 1./min_population_size - 1./max_population_size}, dim=2) # I_0 (1/population_size)

s_set.generate_samples()

lam = s_set.samples # create a pointer for ease of reference later with plotting.

### Visualize Prior 

In [ ]:
# widgets.interactive(pltdata, data = fixed(lam), inds = fixed(None), 
#                     N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
#                     eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="orange", continuous_update=False),
#                     view_dim_1 = widgets.IntSlider(value=0, min=0, max=input_dim-1, step=1, continuous_update=False), 
#                     view_dim_2 = widgets.IntSlider(value=input_dim-1, min=0, max=input_dim-1, step=1, continuous_update=False))

---
# Compute Data Space $O(\Lambda) = \mathcal{D}$ 

Format: `(n_dims, n_samples)`  

In [ ]:
S_t, I_t, R_t = SIR() # t as a subscript stands for "truth"
obs_data_I = I_t + np.random.randn(int(num_observations))*sd_I
obs_data_R = R_t + np.random.randn(int(num_observations))*sd_R

if observables == 2:
    if output_dim == 1: # concatenate the Chi^2 data.
        def PtO_fun(inputs): # that conforms to our desired model input
            predictions = epidemic_IR(inputs)
            M = len(obs_data_I)
            assert predictions.shape[2] == len(obs_data_I)
            residuals_I = predictions[0,:,:] - obs_data_I
            residuals_R = predictions[1,:,:] - obs_data_R
            temp = np.hstack([(residuals_R/sd_R)**2, (residuals_I/sd_I)**2])
#             QoI = (1./(2*M))*np.sum(temp, axis=1).reshape(inputs.shape[0],1)
            QoI = (1./(2*M))*(np.sum( (residuals_R/sd_R)**2, axis=1 ) + np.sum( (residuals_I/sd_I)**2, axis=1 ))
            return QoI
        
    else: # output_dim = 2, treat as dual-Chi^2
        def PtO_fun(inputs): # that conforms to our desired model input
            predictions = epidemic_IR(inputs)
            M = len(obs_data_I)
            assert predictions.shape[2] == len(obs_data_I)
            residuals_I = predictions[0,:,:] - obs_data_I
            residuals_R = predictions[1,:,:] - obs_data_R
            
            QoI_R = (1./M)*np.sum((residuals_R/sd_R)**2, axis=1 ) 
            QoI_I = (1./M)*np.sum( (residuals_I/sd_I)**2, axis=1 )
            return np.array([QoI_I, QoI_R]).T
else: # one observable
    PtO_fun = SSE_generator(epidemic_I, obs_data_I, sd_I)

In [ ]:
p_set = sample.map_samples_and_create_problem(s_set, PtO_fun)
D = p_set.output.samples

# This is how we handle trying to infer the dimension based on what the map put out.
# You can delete this once you are certain your model is correctly defined.
try:
    output_dim = D.shape[1] # if your function was coded correctly, you should have an (n, d) data space.
except IndexError:
    print(Warning("Warning: Your map might be returning the wrong dimensional data."))
    try:
       output_dim = D.shape[0] 
    except IndexError:
        print(Warning("Warning: Guessing it's 1-dimensional."))
        output_dim = 1
print('dimensions :  lambda = '+str(lam.shape)+'   D = '+str(D.shape) )

# Compute Push-Forward of the Prior $P_{O(\Lambda)}$
_ ... i.e. Characterize the Data Space_

In [ ]:
# Interactive Marginal Visualization
p_set.compute_pushforward_dist(method='g')
pf_dist = p_set.pushforward_dist
# pf_dist.kde_object.set_bandwidth(1E-10)
# %matplotlib inline

## Visualize Pushforward of Prior

In [ ]:
# widgets.interactive(pltdata, data = fixed(pf_dist), inds = fixed(None), 
#         N = widgets.IntSlider(value=10000, min = 100, max=10000, step=100, continuous_update=False), 
#         eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="brown", continuous_update=False),
#         view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
#         view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))

# Define Observed Probability Measure $P_\mathcal{D}$

In [ ]:
if observables == 1: # you get to observe I
    p_set.set_observed_dist('gamma', {'a':num_observations/2, 'scale':2/num_observations}, dim=0)
else: # you get to observe R as well
    if output_dim == 1:
        p_set.set_observed_dist('gamma', {'a':(2*num_observations)/2, 'scale':2/(2*num_observations)}, dim=0)
    else:
        p_set.set_observed_dist('gamma', {'a':num_observations/2, 'scale':2/num_observations}, dim=0)
        p_set.set_observed_dist('gamma', {'a':num_observations/2, 'scale':2/num_observations}, dim=1)

obs_dist = p_set.observed_dist # this is define a pointer for ease of reference.

## Visualize Observed Density

In [ ]:
# widgets.interactive(pltdata, data = fixed(obs_dist), inds = fixed(None), 
#         N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
#         eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="wine", continuous_update=False),
#         view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
#         view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))
if output_dim == 1:
    xx = np.linspace(-10,100,1000).reshape(-1,1)
    obs_dist = p_set.observed_dist # this is define a pointer for ease of reference.
    zz = pf_dist.pdf(xx)
    yy = obs_dist.pdf(xx)
    plt.plot(xx, zz, c='blue',label='pf_prior')
    plt.plot(xx, yy,c='orange',label='observed')
    plt.ylim([0,0.2])
    plt.legend()
    plt.show()

---

At this point we have performed the computations we need to. We have evaluated the input points through our map and performed a KDE on them. It would be useful at this point to save this object and/or its evaluation at every point in the data space for later re-use. Doing so here would be an appropriate place. 

--- 

# Accept/Reject Sampling of Posterior

Since we have already used the samples in our prior to compute the pushforward density, we can re-use these with an accept/reject algorithm to get a set of samples generated from the posterior according to the solution of the stochastic inverse problem as outlined in the Consistent Bayes formulation. 

In [ ]:
p_set.set_ratio()
eta_r = p_set.ratio
solve.problem(p_set)

In [ ]:
accept_inds = p_set.accept_inds
lam_accept = p_set.input.samples[accept_inds,:]
num_accept = len(accept_inds)
print('Number accepted: %d = %2.2f%%'%(num_accept, 100*np.float(num_accept)/num_samples))

## Visualize Posterior Density
### (Visualize Accept/Reject Samples)


In [ ]:
if num_accept >= 2:
    display(widgets.interactive(pltdata, data = fixed(lam), inds = fixed(accept_inds), 
            N = widgets.IntSlider(value=num_accept/2, min = 2, max=num_accept, step=1, continuous_update=False), 
            eta_r = fixed(None), space=fixed(0.05), svd=widgets.Checkbox(value=False), color=widgets.Text(value="orange", continuous_update=False),
            view_dim_1 = widgets.IntSlider(value=0, min=0, max=input_dim-1, step=1, continuous_update=False), 
            view_dim_2 = widgets.IntSlider(value=input_dim-1, min=0, max=input_dim-1, step=1, continuous_update=False)))
else:
    raise(Warning("not enough samples accepted."))
# You will visualize the accepted samples in a subset of size N of the input samples. 
# This is mostly for faster plotting, but also so you can see the progression of accepted sampling in the algorithm.


In [ ]:
# np.save('sir_save_dual.npy',p_set)
# np.save('sir_save_comb.npy',p_set)

---
# Now what? 

Well, we can...

# Alternative Visualization Approach

Now that we have constructed the posterior distribution, we can evaluate it at whatever set of samples we want, so we plot it directly, rather than use Accept/Reject.



In [ ]:
nbins = 250

mins = np.array([0.1, 0]) # original domain
maxs = np.array([0.3, 0.2])

X = ngrid(nbins, mins, maxs)
post_eval = p_set.evaluate_posterior(X)
# post_eval = p_set.compute_ratio(PtO_fun(X))*p_set.input.dist.pdf(X)

In [ ]:
# Make the plot
xi = X[:,0].reshape(nbins, nbins)
yi = X[:,1].reshape(nbins, nbins)
zi = post_eval.reshape(nbins, nbins)
plt.pcolormesh(xi, yi, zi)
problem = 'SIR'
ftype = 'png'

plt.scatter([0.2], [0.1], c='white', edgecolor='black', s=100, label='True Parameter')
plt.ylabel('$\\beta$')
plt.xlabel('$\\gamma$')
if observables == 1:
    plt.title('%d Observations of Infected Individuals'%(num_observations))
else:
    plt.title('%d Observations of Infected and Recovered Individuals'%(num_observations))
save_str = '%s_M%d_%dE_obs%d'%(problem.upper(), num_observations, 1000*sd_R, observables)
save_str = 'saveimgs/%s/%s.%s'%(problem.upper(), save_str, ftype)
print("saving", save_str)
plt.savefig(save_str)
plt.show()
# ?plt.pcolormesh

In [ ]:
rat = p_set.compute_ratio(PtO_fun(X))


In [ ]:
inddd = solve.perform_accept_reject(X, rat)

In [ ]:
D = epidemic_IR(X[inddd,:])
nacc = len(X[inddd,:])

In [ ]:
for i in range(nacc):
    plt.plot(t, D[0, i, :], alpha=1.0/n, c='r') # I
    plt.plot(t, D[1, i, :], alpha=1.0/n, c='k') # R
#     plt.plot(t, 1 - data[0, i, :] - data[1, i, :], c='b', alpha=2/n) # S
# plt.plot(t, 1-I-R, c='b', lw=3, ls=':')
plt.plot(t, I, c='r', lw=3, ls='-', label="Infected")
plt.plot(t, R, c='k', lw=3, ls='-', label="Recovered")
plt.xlabel('Time (days)', fontsize=28)
plt.ylabel('Proportion of Population', fontsize=28)
plt.xlim([0,365])
plt.ylim([0,1])
plt.legend(fontsize=18)
plt.title('%d Observations, %d Observables, Pushforward of Posterior'%(2160, observables), fontsize=24)
plt.savefig('saveimgs/SIR/SIR_signal_pf%d_%d.png'%(2160, observables))
# plt.title('%d Observations, %d Observables, Pushforward of Posterior'%(num_observations, observables), fontsize=24)
# plt.savefig('saveimgs/SIR/SIR_signal_pf%d_%d.png'%(num_observations, observables))
plt.show()

[GO BACK TO TOP](#Define-Functions)

In [ ]:
A = X[inddd,:]

In [ ]:
for i in range(A.shape[0]):
    plt.plot(range(A.shape[1]),  A[i,:] , alpha = 3./100)
# for j in range(A.shape[1]):
#     plt.vlines(j, 0, 1)
plt.show()

## _Visualize the Quality of our SIP Solution by Comparing it to the Observed_
_We compare the push-forward of the posterior using accepted samples against the observed density_  
_(SIP = Stochastic Inverse Problem)_
### Observed:

In [ ]:
# widgets.interactive(pltdata, data = fixed(obs_dist), inds = fixed(None), 
#         N = widgets.IntSlider(value=500, min = 100, max=5000, step=100, continuous_update=False), 
#         eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="wine", continuous_update=False),
#         view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
#         view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))

## Pushforward of Posterior:

In [ ]:
# widgets.interactive(pltdata, data = fixed(D), inds = fixed(accept_inds), 
#         N = widgets.IntSlider(value=num_accept/2, min = 2, max=num_accept-1, step=1, continuous_update=False), 
#         eta_r = fixed(None), space=fixed(0.05), svd=fixed(False), color=widgets.Text(value="eggplant", continuous_update=False),
#         view_dim_1 = widgets.IntSlider(value=0, min=0, max=output_dim-1, step=1, continuous_update=False), 
#         view_dim_2 = widgets.IntSlider(value=output_dim-1, min=0, max=output_dim-1, step=1, continuous_update=False))

## Verify our parametric test statistics.
Let's see if the pushforward of the posterior results in a sample mean and standard deviation that are close to the observed.

In [ ]:
# [np.mean(D[accept_inds,:]), np.std(D[accept_inds,:])]

In [ ]:
# [p_set.observed_dist.mean(), p_set.observed_dist.std()]

Now go back to the [Define PtO Map Section](#Define-Your-Parameter-to-Observables-Map) and change the distribution on the prior or choose another example problem.

In [ ]:
# Plotting the default IR results from the epidemic_IR function.
# if plot_new:
#     u = model(new_accepted_in)
#     for i in range(u.shape[0]):
#         d = u[i,:]
#         if i==1:
#             plt.plot(t, d, c='k', alpha=0.1, lw=1, label='New Accept')
#         else:
#             plt.plot(t, d, c='k', alpha=0.1, lw=1)

